In [23]:
%cd /aincome

/aincome


In [24]:
!pwd

/aincome


In [25]:
!ls

Dockerfile_cpu	README.md  docker-compose.yml  ngc	ngccli_mac.zip
Dockerfile_gpu	data	   docs		       ngc.md5	note


In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

#from tqdm import tqdm
#from tqdm.notebook import tqdm
from tqdm import tqdm_notebook as tqdm

import warnings
import typing
from typing import Union, List, Dict, Any, Optional

import gym
import gym_anytrading
from gym_anytrading.envs import TradingEnv, ForexEnv, StocksEnv, Actions, Positions
from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL

from stable_baselines.bench import Monitor

from stable_baselines.common.vec_env import VecEnv, sync_envs_normalization, DummyVecEnv
from stable_baselines.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import PPO2
from stable_baselines import ACKTR
from stable_baselines import A2C
from stable_baselines.common.callbacks import CallbackList, CheckpointCallback, EvalCallback, EventCallback
from stable_baselines.common.callbacks import BaseCallback
from stable_baselines.common.evaluation import evaluate_policy

import quantstats as qs
import mplfinance as mpf

import talib
from yahoo_finance_api2 import share
import yfinance
from Historic_Crypto import Cryptocurrencies
from Historic_Crypto import HistoricalData

import tensorflow as tf

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
!pip list

Package                          Version
-------------------------------- -------------------
absl-py                          0.10.0
alabaster                        0.7.12
apex                             0.1
appdirs                          1.4.4
argon2-cffi                      20.1.0
ascii-graph                      1.5.1
astor                            0.8.1
async-generator                  1.10
atari-py                         0.2.6
attrs                            20.2.0
audioread                        2.1.9
Babel                            2.8.0
backcall                         0.2.0
beautifulsoup4                   4.9.3
bleach                           3.2.1
blis                             0.4.1
boto3                            1.16.2
botocore                         1.19.2
brotlipy                         0.7.0
catalogue                        1.0.0
certifi                          2020.6.20
cffi                             1.14.3
chardet                          3.0.4
c

In [17]:
# 引数情報
symbol      = 'XLM-USD'          # 通貨レート
granularity = 300              # 何秒置きにデータを取得するか(60, 300, 900, 3600, 21600, 86400) が指定可能
start_date  = '2020-01-01-00-00' # データ取得範囲：開始日
end_date    = '2020-01-29-00-00' # データ取得範囲：終了日

dataset_name = "./data/datasets/v1.0/symbol-{}_granularity-{}_start-{}_end-{}.csv".format(symbol, granularity, start_date, end_date)
dataset_train_name = "./data/datasets/v1.0/symbol-{}_granularity-{}_start-{}_end-{}_train.csv".format(symbol, granularity, start_date, end_date)
dataset_valid_name = "./data/datasets/v1.0/symbol-{}_granularity-{}_start-{}_end-{}_valid.csv".format(symbol, granularity, start_date, end_date)


In [18]:
# 過去データが取得可能な通貨レートを確認
crypto_data = Cryptocurrencies().find_crypto_pairs()

Connected to the CoinBase Pro API.
Extended Output is False, returning reduced dataset.


In [19]:
# 出力
print(crypto_data)

            id display_name  fx_stablecoin max_slippage_percentage  status
0      BCH-GBP      BCH/GBP          False              0.03000000  online
1    SUSHI-GBP    SUSHI/GBP          False              0.03000000  online
2      BAT-BTC      BAT/BTC          False              0.03000000  online
3     CGLD-GBP     CGLD/GBP          False              0.03000000  online
4      KRL-USD      KRL/USD          False              0.05000000  online
..         ...          ...            ...                     ...     ...
573  LINK-USDT    LINK/USDT          False              0.03000000  online
574  BICO-USDT    BICO/USDT          False              0.03000000  online
575   AAVE-USD     AAVE/USD          False              0.03000000  online
576    AST-USD      AST/USD          False              0.03000000  online
577   IOTX-USD     IOTX/USD          False              0.03000000  online

[578 rows x 5 columns]


In [20]:
data = HistoricalData(symbol,granularity,start_date,end_date).retrieve_data()

Checking input parameters are in the correct format.
Formatting Dates.
Checking if user supplied is available on the CoinBase Pro API.
Connected to the CoinBase Pro API.
Ticker 'XLM-USD' found at the CoinBase Pro API, continuing to extraction.
Provisional Start: 2020-01-01T00:00:00
Provisional End: 2020-01-02T01:00:00
Data for chunk 1 of 27 extracted
Provisional Start: 2020-01-02T01:00:00
Provisional End: 2020-01-03T02:00:00
Data for chunk 2 of 27 extracted
Provisional Start: 2020-01-03T02:00:00
Provisional End: 2020-01-04T03:00:00
Data for chunk 3 of 27 extracted
Provisional Start: 2020-01-04T03:00:00
Provisional End: 2020-01-05T04:00:00
Data for chunk 4 of 27 extracted
Provisional Start: 2020-01-05T04:00:00
Provisional End: 2020-01-06T05:00:00
Data for chunk 5 of 27 extracted
Provisional Start: 2020-01-06T05:00:00
Provisional End: 2020-01-07T06:00:00
Data for chunk 6 of 27 extracted
Provisional Start: 2020-01-07T06:00:00
Provisional End: 2020-01-08T07:00:00
Data for chunk 7 of 27 ext

In [26]:
#### Check data
data.head(5)

#### Adjust data

data2 = data.reset_index()
data2.columns = ["Time", "Low", "High",  "Open", "Close", "volume"]

data2.head(5)

# Save dataset
data2.to_csv(dataset_name)

In [27]:
df_dataset = pd.read_csv(dataset_name, index_col=0)

In [28]:
df_dataset

,Time,Low,High,Open,Close,volume
0,2020-01-01 00:05:00,0.044725,0.044798,0.044725,0.044797,4827
1,2020-01-01 00:10:00,0.044739,0.044806,0.044806,0.044798,7185
2,2020-01-01 00:15:00,0.044715,0.044798,0.044798,0.044777,13650
3,2020-01-01 00:20:00,0.044701,0.044769,0.044769,0.044725,52259
4,2020-01-01 00:25:00,0.044700,0.044782,0.044766,0.044704,89821
...,...,...,...,...,...,...
8049,2020-01-28 23:40:00,0.060641,0.060912,0.060912,0.060755,39887
8050,2020-01-28 23:45:00,0.060577,0.060846,0.060824,0.060703,119475
8051,2020-01-28 23:50:00,0.060448,0.060825,0.060448,0.060709,51945
8052,2020-01-28 23:55:00,0.060678,0.060888,0.060831,0.060678,120292


In [29]:
df_dataset["Time"] = pd.to_datetime(df_dataset["Time"])
df_dataset = df_dataset.set_index("Time")
print(df_dataset)

                          Low      High      Open     Close  volume
Time                                                               
2020-01-01 00:05:00  0.044725  0.044798  0.044725  0.044797    4827
2020-01-01 00:10:00  0.044739  0.044806  0.044806  0.044798    7185
2020-01-01 00:15:00  0.044715  0.044798  0.044798  0.044777   13650
2020-01-01 00:20:00  0.044701  0.044769  0.044769  0.044725   52259
2020-01-01 00:25:00  0.044700  0.044782  0.044766  0.044704   89821
...                       ...       ...       ...       ...     ...
2020-01-28 23:40:00  0.060641  0.060912  0.060912  0.060755   39887
2020-01-28 23:45:00  0.060577  0.060846  0.060824  0.060703  119475
2020-01-28 23:50:00  0.060448  0.060825  0.060448  0.060709   51945
2020-01-28 23:55:00  0.060678  0.060888  0.060831  0.060678  120292
2020-01-29 00:00:00  0.060581  0.060888  0.060692  0.060581   30653

[8054 rows x 5 columns]


In [30]:
df_train = df_dataset[0:int(len(df_dataset)*0.7)]

In [31]:
df_train

,Low,High,Open,Close,volume
Time,,,,,
2020-01-01 00:05:00,0.044725,0.044798,0.044725,0.044797,4827
2020-01-01 00:10:00,0.044739,0.044806,0.044806,0.044798,7185
2020-01-01 00:15:00,0.044715,0.044798,0.044798,0.044777,13650
2020-01-01 00:20:00,0.044701,0.044769,0.044769,0.044725,52259
2020-01-01 00:25:00,0.044700,0.044782,0.044766,0.044704,89821
...,...,...,...,...,...
2020-01-20 14:05:00,0.062681,0.062763,0.062735,0.062685,33943
2020-01-20 14:10:00,0.062595,0.062784,0.062652,0.062702,47243
2020-01-20 14:15:00,0.062102,0.062686,0.062614,0.062407,399058


In [32]:
df_test = df_dataset[int(len(df_dataset)*0.7):]

In [33]:
df_test

,Low,High,Open,Close,volume
Time,,,,,
2020-01-20 14:30:00,0.062597,0.062759,0.062753,0.062722,64223
2020-01-20 14:35:00,0.062631,0.062782,0.062665,0.062741,23331
2020-01-20 14:40:00,0.062741,0.063229,0.062741,0.063229,174560
2020-01-20 14:45:00,0.063154,0.063413,0.063212,0.063402,42457
2020-01-20 14:50:00,0.063305,0.063752,0.063325,0.063535,182846
...,...,...,...,...,...
2020-01-28 23:40:00,0.060641,0.060912,0.060912,0.060755,39887
2020-01-28 23:45:00,0.060577,0.060846,0.060824,0.060703,119475
2020-01-28 23:50:00,0.060448,0.060825,0.060448,0.060709,51945


In [34]:
## Create technical indicator func
def create_technical_indicator(_df):

    df = _df.copy()
    """
    # ---------
    # SMA
    df['SMA'] = talib.SMA(df['Close'], timeperiod=5)

    # ---------
    # EMA
    df['EMA'] = talib.EMA(df['Close'], timeperiod=5)

    # ---------
    # BB
    df["BB_UPPER"], df["BB_MIDDLE"], df["BB_LOWER"] = talib.BBANDS(df["Close"], timeperiod=20, matype=talib.MA_Type.EMA)

    # ---------
    # RSI
    df["RSI"] = talib.RSI(df["Close"], timeperiod=5)

    # ---------
    # stochastics
    df["SLOW_K"], df["SLOW_D"] = talib.STOCH(df["High"], df["Low"], df["Close"], fastk_period=5, slowk_period=3, slowd_period=3)

    # ---------
    # DMI
    df["DMI_p_DI"] = talib.PLUS_DI(df["High"], df["Low"], df["Close"], timeperiod=14)
    df["DMI_m_DI"] = talib.MINUS_DI(df["High"], df["Low"], df["Close"], timeperiod=14)
    df["DMI_ADX"] = talib.ADX(df["High"], df["Low"], df["Close"], timeperiod=14)

    # ---------
    # MACD
    df["MACD"], df["SIGNAL"], df["HIST"] = talib.MACD(df["Close"], fastperiod=12, slowperiod=26, signalperiod=9)
    """

    df.dropna(inplace=True)
    return df

In [35]:
df_train_ti = create_technical_indicator(df_train)
df_test_ti = create_technical_indicator(df_test)

In [36]:
df_train_ti.head(5)

,Low,High,Open,Close,volume
Time,,,,,
2020-01-01 00:05:00,0.044725,0.044798,0.044725,0.044797,4827
2020-01-01 00:10:00,0.044739,0.044806,0.044806,0.044798,7185
2020-01-01 00:15:00,0.044715,0.044798,0.044798,0.044777,13650
2020-01-01 00:20:00,0.044701,0.044769,0.044769,0.044725,52259
2020-01-01 00:25:00,0.044700,0.044782,0.044766,0.044704,89821


In [37]:
df_train_ti.to_csv(dataset_train_name)
df_test_ti.to_csv(dataset_valid_name)